# 필작이의 chatbot




In [ ]:
# openai, streamlit, localtunnel 설치
!pip install openai==0.28
!pip install streamlit
!npm install localtunnel
!pip install -U typing_extensions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.67s

3 packages are looking for funding
  run `n

In [ ]:


%%writefile elastic_gpt_app.py


import os
import streamlit as st
import openai
import json
from elasticsearch import Elasticsearch

#openai apy key 입력
openai_api = ''

openai.api_key = openai_api
model = "gpt-3.5-turbo-0301"

def es_connect(cid, user, passwd):
    es = Elasticsearch(cloud_id=cid, http_auth=(user, passwd))
    return es

# Elasticsearch Search
def search(query_text):
    # ES Cloud ID, Username, Password 입력
    cid = ''
    cu = ''
    cp = ''

    es = es_connect(cid, cu, cp)

    # 엘라스틱 검색
    query = {
        "bool": {
          "should": [{
            "match": {
              "INDIV_RAWMTRL_NM": {
                "query": query_text,
                "boost": 50
              }
            }
          },
          {
          "match_phrase_prefix": {
            "INDIV_RAWMTRL_NM": {
              "query": query_text,
              "boost": 1
            }
          }
          }
      ],
      "filter": [{
        "exists": {
          "field": "INDIV_RAWMTRL_NM"
          }
      }],
            "minimum_should_match": 1
        }
    }
    sort={"_score": {"order": "desc"}}
    index = 'origin_pills'
    fields = ["PRDLST_NM", "PRIMARY_FNCLTY", "IFTKN_ATNT_MATR_CN", "INDIV_RAWMTRL_NM" ]

    resp = es.search(index=index,
                     query=query,
                     fields=fields,
                     size=3, # 상위 3개의 문서 가져옴
                     sort=sort,
                     source=False
                     )
    names = []
    funcs = []
    warnings = []
    ingreds = []


    if 'hits' in resp and 'hits' in resp['hits'] and resp['hits']['hits']:
      for hit in resp['hits']['hits']:
          name = hit['fields']['PRDLST_NM'][0]
          func = hit['fields']['PRIMARY_FNCLTY'][0]
          warning = hit['fields']['IFTKN_ATNT_MATR_CN'][0]
          ingred = hit['fields']['INDIV_RAWMTRL_NM'][0]

          names.append(name)
          funcs.append(func)
          warnings.append(warning)
          ingreds.append(ingred)

    return names, funcs, warnings, ingreds




def truncate_text(text, max_tokens):
    tokens = text.split()
    if len(tokens) <= max_tokens:
        return text

    return ' '.join(tokens[:max_tokens])

# Chat_gpt 답변 생성
def chat_gpt(prompt, model="gpt-3.5-turbo", max_tokens=1024, max_context_tokens=4000, safety_margin=5):
    # Truncate the prompt content to fit within the model's context length
    truncated_prompt = truncate_text(prompt, max_context_tokens - max_tokens - safety_margin)

    response = openai.ChatCompletion.create(model=model,
                                            messages=[{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": truncated_prompt}])

    return response["choices"][0]["message"]["content"]


# 입력창
with st.form("chat_form"):
    query = st.text_input("You: ") # 사용자에게 질문 받기
    submit_button = st.form_submit_button("Send")

# 답변 출력
negResponse = "해당 질문에 관련된 건강기능식품 정보가 없습니다."
if submit_button:
    prompt = f"다음 질문과 관련있는 영양성분을 찾아주세요. 찾은 영양성분을 적어, 답변을 파이썬의 리스트 형태 []로 주세요. : {query}"
    answer = chat_gpt(prompt)
    names, funcs, warnings, ingreds = search(answer)
    prompt2 = f"'{query}'에 대한 답을 다음 정보를 활용해 사람이 말하는 것처럼 말하세요. 활용 시에는 제품을 추천하되 답과 관련 없는 말은 하지 마세요. 또한 왜 '{query}'에 대한 답인지 자세하고 논리적으로 설명하세요: 품목명-{names}, 주요 성분-{ingreds}, 섭취시주의사항-{warnings}. 추가적으로 다음의 정보를 참고할 수 있습니다. 해당 정보는 관련이 없다면 안 사용해도 됩니다.:주된기능성-{funcs} \n제공된 모든 문서로 답변을 할 수 없는 경우 '{negResponse}'로 응답하십시오."
    answer2 = chat_gpt(prompt2) # 출력해야 하는 chatbot 답변
    st.write(f"필작이: {answer2}")



Writing elastic_gpt_app.py


In [ ]:
!pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.1/432.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.2 MB/s eta 0:00:00


## Web UI(제대로 출력되는지 확인하는 파트)

In [ ]:
# Streamlit 실행
!streamlit run /content/elastic_gpt_app.py &>/content/logs.txt &

```python
🐘
""
UI를 콘솔이 아닌 웹에서 보기 위해 임시 페이지를 열었어요.
아래에서 확인되는 Endpoint IP를 복사하고, npx에서 출력해주는 임시 URL에 들어가 입력해 주세요!
""
```

In [ ]:
# Endpoint IP 출력
import urllib
print("IP Endpoint:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

IP Endpoint: 34.106.213.7


In [ ]:
# 임시 URL 발급
!npx localtunnel

npx: installed 22 in 2.81s
your url is: https://fresh-drinks-bake.loca.lt
^C
